In [ ]:
### Manually consuming an iterator

In [ ]:
lst = iter(list(range(10)))

try:
    while True:
        print(next(lst))
except StopIteration as si:
    pass

In [ ]:
# or without try

lst = iter(range(10))

while True:
    line = next(lst, None)  # we are telling next to return None when its done iterating 
    if line is None:
        break
    print(line)

### Using a file iterator

In [ ]:
with open("/etc/passwd", "r") as f:
    # note: f is a generator
    try:
        while True:
            entry = next(f)
            print(entry)
    except StopIteration as si:
        pass

In [ ]:
# instructing next to do nothing, passing None when there is nothing more to iterate on
lst = iter(list(range(10)))
while True:
    entry = next(lst, None) # passing None prevents from throwing StopIteration exception
    print(entry, end = ' ')
    if entry is None:
        break
    